In [1]:
import json
import time
import os
import numpy as np

from cortexsubsetloader import CortexSubsetLoader

In [2]:
past_datasets = []
data_loc = 'data'
datasets = os.listdir(data_loc)
most_recent = 0
for f in datasets:
    if 'cortex_' in f and '.json' in f:
        version = int(f.split('cortex_')[-1].split('_')[0].replace('e', ''))
        if version > most_recent:
            most_recent = version
        past_datasets.append(f'{data_loc}/{f}')

def load_old_data(past_datasets):
    global most_recent, agg_old_data
    agg_old_data = []
    for dataset in past_datasets:
        with open(dataset, 'r') as f:
            agg_old_data = agg_old_data + json.load(f)
    return agg_old_data

agg_old_data = [] # load_old_data(past_datasets)
saveable_dataset = []
version_num = most_recent + 1

def save_dataset(version, data, reload_agg=True):
    global past_datasets, agg_old_data
    save_len = (len(data))
    with open(f'data/cortex_{version}_{save_len}.json', 'w') as f:
        json.dump(data[-save_len:], f)
    past_datasets.append(f'data/cortex_{version}_{save_len}.json')
    if reload_agg: agg_old_data = load_old_data(past_datasets)
    print(f'Saved {save_len} samples to data/cortex_{version}_{save_len}.json')


print(f"found {len(past_datasets)} datasets, comprised of {len(agg_old_data)} samples")
print(f"most recent dataset is {most_recent}")

found 8 datasets, comprised of 0 samples
most recent dataset is 686


In [3]:
def make_new_eval_set(size=512):
    global version_num
    eval_set = []
    cortex_subset_loader = CortexSubsetLoader(latest=True, random_seed = None, max_samples=2560, steps=1, progress=True, running=True,
                    retry_limit=5, page_size=400, retry_delay=5)
    for prompt, response in cortex_subset_loader.buffer:
            new = {"instruction": prompt, "response": response}
            if new not in saveable_dataset and new not in agg_old_data and new not in eval_set:
                eval_set.append(new)
    eval_set = np.array(eval_set)[np.random.permutation(len(eval_set))].tolist()

    if len(eval_set) > size:
         save_dataset('e'+str(version_num), eval_set[:size])
         version_num += 1
    else:
        print(f"only found {len(eval_set)} new samples, not saving")


def create_new_datasets(num_new=1, size=2048*4, sleep_minutes=4, max_samples_per_step=1024, ignore_old=False, steps=1):
    global version_num, saveable_dataset
    datasets_so_far = 0
    while datasets_so_far < num_new:
        cortex_subset_loader = CortexSubsetLoader(latest=True, random_seed = None, max_samples=max_samples_per_step, steps=steps, 
                                        progress=True, running=True, retry_limit=5, page_size=400, retry_delay=5)
        print("samples collected:",len(cortex_subset_loader.buffer))
        current_size = len(saveable_dataset)
        for prompt, response in cortex_subset_loader.buffer:
            new = {"instruction": prompt, "response": response}
            if new not in saveable_dataset:
                if ignore_old or new not in agg_old_data:
                    saveable_dataset.append(new)
        samples_added = len(saveable_dataset) - current_size
        print("samples collected:",len(cortex_subset_loader.buffer), "\tdeduped samples added:", samples_added)
        saveable_dataset = np.array(saveable_dataset)
        sp = np.random.permutation(len(saveable_dataset))
        saveable_dataset = saveable_dataset[sp].tolist()
        print(len(saveable_dataset))

        if len(saveable_dataset) >= size:
            save_dataset(version_num, saveable_dataset[:size], reload_agg=not ignore_old)
            version_num += 1
            saveable_dataset = saveable_dataset[size:]
            datasets_so_far += 1

        if datasets_so_far < num_new:
            time.sleep(sleep_minutes * 60)

In [4]:
data = []
CortexSubsetLoader(latest=True, random_seed = None, max_samples=512, steps=1, 
                                        progress=True, running=True, retry_limit=5, page_size=400, retry_delay=5, ignore_list=data)

In [6]:
make_new_eval_set(size=512)

Did not collect 2560, only got 2318
Saved 512 samples to data/cortex_e687_512.json


In [22]:
create_new_datasets(num_new=48, size=8192+512, max_samples_per_step=4096, ignore_old=True, sleep_minutes=5, steps=1)

Did not collect 4096, only got 2036
samples collected: 2036
samples collected: 2036 	deduped samples added: 2036
2036


KeyboardInterrupt: 

In [7]:
create_new_datasets(num_new=48, size=8144+512, max_samples_per_step=3072, ignore_old=True, sleep_minutes=5)

Did not collect 3072, only got 2376
samples collected: 2376
samples collected: 2376 	deduped samples added: 160
2591


Did not collect 3072, only got 2205
samples collected: 2205
samples collected: 2205 	deduped samples added: 425
3016


Did not collect 3072, only got 2213
samples collected: 2213
samples collected: 2213 	deduped samples added: 219
3235


Did not collect 3072, only got 2170
samples collected: 2170
samples collected: 2170 	deduped samples added: 176
3411


Did not collect 3072, only got 2189
samples collected: 2189
samples collected: 2189 	deduped samples added: 195
3606


...

Did not collect 3072, only got 1738
samples collected: 1738
samples collected: 1738 	deduped samples added: 393
5507


Did not collect 3072, only got 1736
samples collected: 1736
samples collected: 1736 	deduped samples added: 212
5719


Did not collect 3072, only got 1522
samples collected: 1522
samples collected: 1522 	deduped samples added: 222
5941


Did not collect 3072, only got 1735
samples collected: 1735
samples collected: 1735 	deduped samples added: 328
6269


Did not collect 3072, only got 1523
samples collected: 1523
samples collected: 1523 	deduped samples added: 158
6427


In [ ]:
create_new_datasets(num_new=48, size=6144+512, max_samples_per_step=1600, ignore_old=True, sleep_minutes=5)

samples collected: 1600
samples collected: 1600 	deduped samples added: 1600
1600


samples collected: 1600
samples collected: 1600 	deduped samples added: 1190
2790


Did not collect 1600, only got 1483
samples collected: 1483
samples collected: 1483 	deduped samples added: 860
3650


samples collected: 1600
samples collected: 1600 	deduped samples added: 1174
4824


samples collected: 1600
samples collected: 1600 	deduped samples added: 1181
6005


...

Did not collect 1600, only got 1435
samples collected: 1435
samples collected: 1435 	deduped samples added: 390
4848


Did not collect 1600, only got 1445
samples collected: 1445
samples collected: 1445 	deduped samples added: 601
5449


Did not collect 1600, only got 1430
samples collected: 1430
samples collected: 1430 	deduped samples added: 401
5850


Did not collect 1600, only got 1449
samples collected: 1449
samples collected: 1449 	deduped samples added: 416
6266


Did not collect 1600, only got 1410
samples collected: 1410
samples collected: 1410 	deduped samples added: 566
6832
Saved 6656 samples to data/cortex_528_6656.json


In [2]:
with open('data/cortex_43_131072.json', 'r') as f:
    big_dataset = json.load(f)

In [3]:
big_dataset[0]

{'instruction': 'How does exposure to toxic substances impact human health over time?',
 'response': 'Exposure to toxic substances can have a wide range of effects on human health, and the impact over time can vary greatly depending on the type of substance, the level of exposure, the duration of exposure, and individual susceptibility. Here are some of the ways that toxic substances can impact health over time:\n\n1. **Acute Effects**: Some toxic substances can cause immediate health effects, such as irritation, nausea, dizziness, or acute poisoning, which can be severe and even life-threatening.\n\n2. **Chronic Effects**: Long-term exposure to certain toxins can lead to chronic health conditions. For example, exposure to asbestos can cause mesothelioma, a type of lung cancer, and exposure to lead can result in neurological damage and cognitive impairments.\n\n3. **Cancer**: Many toxic substances are carcinogens, meaning they can cause cancer. The development of cancer typically invol

In [4]:
big_sentences = []
for conv in big_dataset:
    big_sentences.append(conv['instruction'] + ' ' + conv['response'])

In [5]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer("all-mpnet-base-v2")

c:\Users\Alex\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [6]:
embeddings = model.encode(big_sentences, show_progress_bar=True, batch_size=64)

Batches:   0%|          | 0/2048 [00:00<?, ?it/s]

In [3]:
# numpy save embeddings
# np.save('data/cortex_43_131072_embeddings.npy', embeddings)
# numpy load embeddings
embeddings = np.load('data/cortex_43_131072_embeddings.npy')

In [4]:
embeddings.shape

(131072, 768)

In [4]:
# run pca to get explained variance
from sklearn.decomposition import PCA
pca = PCA(n_components=256)
pca.fit(embeddings)
print(pca.explained_variance_ratio_.cumsum())

[0.04161603 0.07500422 0.10545201 0.13115078 0.15446855 0.17644582
 0.19645111 0.21355434 0.23053718 0.24664475 0.26172035 0.27602487
 0.28913602 0.3021988  0.31459717 0.32611907 0.33735315 0.3481372
 0.35838842 0.36826377 0.37781801 0.38725323 0.39614332 0.40465277
 0.4129363  0.42091202 0.42873161 0.43625778 0.44357414 0.45072301
 0.45760817 0.46433947 0.47088953 0.47724658 0.4835029  0.48959251
 0.49560119 0.50150837 0.50724732 0.51285522 0.51833798 0.52372669
 0.52904185 0.53419638 0.53929064 0.54432483 0.54928434 0.55407916
 0.55880942 0.56345253 0.56801047 0.57250069 0.57696082 0.58123531
 0.58549071 0.58962495 0.59368873 0.59771049 0.60167826 0.60562206
 0.60954342 0.61338836 0.61719923 0.62091934 0.62460185 0.62822375
 0.63177549 0.63527651 0.63873708 0.64216197 0.64554361 0.64887523
 0.6521141  0.65531605 0.65849627 0.66163976 0.66472084 0.66775257
 0.67075244 0.67370301 0.67662345 0.67952911 0.68242249 0.68529409
 0.68813402 0.69092322 0.69368414 0.6963964  0.69908451 0.70176

In [4]:
from sklearn.cluster import MiniBatchKMeans

In [27]:
kmeans = MiniBatchKMeans(n_clusters=256, max_iter=1000)
kmeans.fit(embeddings)

# get size of each cluster
unique, counts = np.unique(kmeans.labels_, return_counts=True)
percentages = counts / np.sum(counts)
print(counts)
print(unique)
print()
# get percent of total in each cluster, sorted
order = np.argsort(percentages)
print(percentages[order][::-1])

[ 582  474  694 1100  750  442  661  625  555  473  712  818 1108 1073
  874  472  823  454  736  642  426  752  810  318 1201  397  505  609
  952 1022  531  938  494 1879 1586  886  213  555  595  312  588  657
  849  248 1279 1034  558  435  427  863  212  760  292  843  160  355
  349  211  300  340  391  193  165  906  301  785  408  681  350  568
 1056  140  524  197 1642  922  574  280  461  295  241  629  654  816
  179  409 1153  402  522  437  357  407  714   77  752  270  662  186
  782 1037  694  589  308  556  254  122  483  388  399   69  572  615
  421  384  598  176  669  900  505  326  700  374  477 1048  240  374
 1036  544  100  331 1189  729  332  624  267   50  192  460  114 1019
  839  990  324  531  340  803  313  101  480  215 1254 1220  584  358
  332  549  339  185  112  213  686  339  192 1283  693  441  896  600
  955  157  129  207  470  431  521  577  313  676  874  177  415  252
  342  975  177  494  416  103  464  234  396  301  281  561  229  167
 1334 

In [26]:
# create function that derives x 'perfect' batches from the data, where each batch has a number of samples from each cluster according to the percentages
def get_perfect_batches(data, labels, percentages, batch_size, n_batches):
    unique, counts = np.unique(labels, return_counts=True)
    percentages = counts / np.sum(counts)
    n_samples_per_batch_per_cluster = percentages * batch_size
    perfect_batches = []
    for i in range(n_batches):
        batch = []
        for j in range(len(n_samples_per_batch_per_cluster)):
            n = int(np.ceil(n_samples_per_batch_per_cluster[j]))
            boo = labels == unique[j]
            arang = np.arange(len(data))[boo]
            p = np.random.permutation(len(arang))
            cluster_samples_idx = arang[p]
            x = 0
            for sample_idx in cluster_samples_idx:
                sample = data[sample_idx]
                if sample not in batch and sample not in perfect_batches:
                    x += 1
                    batch.append(sample)
                if x >= n:
                    break
        p = np.random.permutation(len(batch))
        batch = np.array(batch)[p[:batch_size]].tolist() # in case we overshot
        perfect_batches += batch
    return np.array(perfect_batches).reshape(n_batches, batch_size, -1)

def get_random_batches(data, batch_size, n_batches):
    random_batches = []
    for i in range(n_batches):
        p = np.random.permutation(len(data))
        batch = data[p[:batch_size]]
        random_batches.append(batch)
    return np.stack(random_batches)

In [30]:
kmeans = MiniBatchKMeans(n_clusters=32, max_iter=1000)
kmeans.fit(embeddings)

# get size of each cluster
unique, counts = np.unique(kmeans.labels_, return_counts=True)
percentages = counts / np.sum(counts)

# test embedding variances for different batch sizes
sample_size = 256
batch_sizes = [1, 2, 4, 8, 16, 32, 64, 128, 256, 512, 1024, 2048]
variances = []
random_variances = []
for batch_size in batch_sizes:
    perfect_batches = get_perfect_batches(embeddings, kmeans.labels_, percentages, batch_size, sample_size)
    random_batches = get_random_batches(embeddings, batch_size, sample_size)
    variances.append(perfect_batches.mean(axis=1).std(axis=0).mean())
    random_variances.append(random_batches.mean(axis=1).std(axis=0).mean())
    print(f"batch size: {batch_size}\tperfect variance: {variances[-1]}\trandom variance: {random_variances[-1]}")

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [15]:
kmeans = MiniBatchKMeans(n_clusters=256, max_iter=1000)
kmeans.fit(embeddings)

# get size of each cluster
unique, counts = np.unique(kmeans.labels_, return_counts=True)
percentages = counts / np.sum(counts)

# test embedding variances for different batch sizes
sample_size = 256
batch_sizes = [1, 2, 4, 8, 16, 32, 64, 128, 256, 512, 1024, 2048]
variances = []
random_variances = []
for batch_size in batch_sizes:
    perfect_batches = get_perfect_batches(embeddings, kmeans.labels_, percentages, batch_size, sample_size)
    random_batches = get_random_batches(embeddings, batch_size, sample_size)
    variances.append(perfect_batches.mean(axis=1).std(axis=0).mean())
    random_variances.append(random_batches.mean(axis=1).std(axis=0).mean())
    print(f"batch size: {batch_size}\tperfect variance: {variances[-1]}\trandom variance: {random_variances[-1]}")

batch size: 1	perfect variance: 0.03315884247422218	random variance: 0.03285451605916023
batch size: 2	perfect variance: 0.023576922714710236	random variance: 0.0232460405677557
batch size: 4	perfect variance: 0.016451319679617882	random variance: 0.01637638360261917
batch size: 8	perfect variance: 0.011713641695678234	random variance: 0.01170428842306137
batch size: 16	perfect variance: 0.008184890262782574	random variance: 0.008278141729533672
batch size: 32	perfect variance: 0.005673990119248629	random variance: 0.005831896793097258
batch size: 64	perfect variance: 0.003845666768029332	random variance: 0.004118447657674551
batch size: 128	perfect variance: 0.002521816873922944	random variance: 0.0029091264586895704
batch size: 256	perfect variance: 0.0016723753651604056	random variance: 0.0020459347870200872
batch size: 512	perfect variance: 0.0011182003654539585	random variance: 0.0014570127241313457
batch size: 1024	perfect variance: 0.0007626183796674013	random variance: 0.001029

In [28]:
new_dataset = get_perfect_batches(big_dataset, kmeans.labels_, percentages, 256, (16384+512)//256)

In [29]:
new_dataset = new_dataset.reshape(-1).tolist()
with open('data/perf_43_16896_256.json', 'w') as f:
    json.dump(new_dataset, f)

In [31]:
new_dataset = get_perfect_batches(big_dataset, kmeans.labels_, percentages, 32, (16384+512)//32)
new_dataset = new_dataset.reshape(-1).tolist()
with open('data/perf_43_16896_32.json', 'w') as f:
    json.dump(new_dataset, f)

In [27]:
from sklearn.cluster import MiniBatchKMeans

def kmeans_percent_attack(n_clusters=2048, n_attack_clusters=128, percent=0.75):
    init = 'k-means++' if n_attack_clusters <= 1024 else 'random'
    n_init = 3 if n_attack_clusters <= 1024 else 10
    kmeans_model = MiniBatchKMeans(n_clusters=n_attack_clusters, init=init, n_init=n_init, batch_size=1024, verbose=True, max_iter=1000)
    kmeans_model.fit(embeddings)
    # print the min max and mean cluster counts
    cluster_counts = np.bincount(kmeans_model.labels_)
    print(f'Min cluster size: {cluster_counts.min()}')
    print(f'Max cluster size: {cluster_counts.max()}')
    print(f'Mean cluster size: {cluster_counts.mean()}')
    # select the n largest clusters that make up percent of the data
    cluster_counts = np.bincount(kmeans_model.labels_)
    sorted_cluster_counts = np.argsort(cluster_counts)

    print(kmeans_model.labels_[:10])
    print(cluster_counts[:10])
    print(sorted_cluster_counts[-10:])


    n = 0
    total = 0
    for i in range(1, len(cluster_counts)):
        total += cluster_counts[sorted_cluster_counts[-i]]
        if total > len(embeddings) * percent:
            n = i
            break
    print(f'Using {n} clusters to make up {percent*100}% of the data, {total} samples')
    largest_clusters = sorted_cluster_counts[-n:]

    # get samples from the largest clusters
    subset_embeds = []
    subset_samples = []
    for c in largest_clusters:
        start = 0
        for boo in kmeans_model.labels_ == c:
            if boo:
                subset_embeds.append(embeddings[start])
                subset_samples.append(big_dataset[start])
            start += 1
    subset_embeds = np.array(subset_embeds)
    subset_samples = np.array(subset_samples)

    # save a random 8192+512 subset of subset_samples
    sp = np.random.permutation(len(subset_samples))
    with open(f'data/atk_43_{8192+512}_0.json', 'w') as f:
        json.dump(subset_samples[sp][:8192+512].tolist(), f)
    if len(subset_samples) > (8192+512)*2:
        with open(f'data/atk_43_{8192+512}_1.json', 'w') as f:
            json.dump(subset_samples[sp][8192+512:8192+512*2].tolist(), f)

    create_kmeans_subset_dataset(n_clusters, subset_samples, subset_embeds, name=f'atk_43_{n_clusters}')
    if n_clusters >= 8192:
        create_kmeans_subset_dataset(2048, subset_samples, subset_embeds, name=f'atk_43_{2048}')
    if n_clusters >= 2048:
        create_kmeans_subset_dataset(512, subset_samples, subset_embeds, name=f'atk_43_{512}')



def create_kmeans_subset_dataset(n_clusters, dataset, embeddings, name='ddc_43'):
    # Initialize the KMeans model
    init = 'k-means++' if n_clusters <= 1024 else 'random'
    n_init = 3 if n_clusters <= 1024 else 10
    kmeans_model = MiniBatchKMeans(n_clusters=n_clusters, init=init, n_init=n_init, batch_size=1024, verbose=True, max_iter=1000)

    # Fit the model to the training embeddings
    kmeans_model.fit(embeddings)

    # print the min max and mean cluster counts
    cluster_counts = np.bincount(kmeans_model.labels_)
    print(f'Min cluster size: {cluster_counts.min()}')
    print(f'Max cluster size: {cluster_counts.max()}')
    print(f'Mean cluster size: {cluster_counts.mean()}')

    # Get the cluster centers
    cluster_centers = kmeans_model.cluster_centers_

    # get the closest sentence to each cluster center
    print('Finding closest sentences to cluster centers')
    closest_sentences = []
    n = 0
    for center in cluster_centers:
        closest = np.argmin(np.linalg.norm(embeddings - center, axis=1))
        closest_sentences.append(dataset[closest])
        n += 1
        if n % (n_clusters  // 32) == 0:
            print('.', end='')


    # save as a new dataset
    with open(f'data/{name}_{n_clusters}.json', 'w') as f:
        json.dump(closest_sentences, f)
    print(f'Saved {n_clusters} samples to data/ddc_43_{n_clusters}.json')

In [22]:
embeddings[[0,1,2,3,4,5,6,7,8,9]]

array([[-0.04811173,  0.02071453,  0.00627764, ..., -0.03249869,
        -0.04462671, -0.01749344],
       [-0.01551625,  0.02464623,  0.02394607, ..., -0.01448496,
         0.02658922,  0.02581609],
       [ 0.03815261,  0.08042033,  0.03174035, ..., -0.03234309,
         0.00742697,  0.01021684],
       ...,
       [ 0.04100377, -0.04117557,  0.00125982, ...,  0.05581007,
         0.0278706 , -0.01357377],
       [ 0.0198206 ,  0.0703216 ,  0.01119775, ...,  0.00825845,
         0.02207336, -0.03420079],
       [ 0.03043545, -0.04029011,  0.0157929 , ...,  0.01468168,
         0.00628204, -0.02334514]], dtype=float32)

In [28]:
kmeans_percent_attack(n_clusters=8192, n_attack_clusters=512, percent=0.75)

Init 1/3 with method k-means++
Inertia for init 1/3: 1797.5601806640625
Init 2/3 with method k-means++
Inertia for init 2/3: 1794.9169921875
Init 3/3 with method k-means++
Inertia for init 3/3: 1757.19921875
[MiniBatchKMeans] Reassigning 128 cluster centers.
Minibatch step 1/128000: mean batch inertia: 0.5463858842849731
Minibatch step 2/128000: mean batch inertia: 0.5274631381034851, ewa inertia: 0.5274631381034851
Minibatch step 3/128000: mean batch inertia: 0.45249855518341064, ewa inertia: 0.5262918254317654
Minibatch step 4/128000: mean batch inertia: 0.45633962750434875, ewa inertia: 0.5251988306780377
Minibatch step 5/128000: mean batch inertia: 0.421335369348526, ewa inertia: 0.5235759764761592
Minibatch step 6/128000: mean batch inertia: 0.40969088673591614, ewa inertia: 0.5217965355250249
Minibatch step 7/128000: mean batch inertia: 0.4173332750797272, ewa inertia: 0.5201643095334633
Minibatch step 8/128000: mean batch inertia: 0.40994003415107727, ewa inertia: 0.518442068370

KeyboardInterrupt: 

In [36]:
n_attack_clusters = 128
percent = 0.75
n_clusters = 8192

init = 'k-means++' if n_attack_clusters <= 1024 else 'random'
n_init = 3 if n_attack_clusters <= 1024 else 10
kmeans_model = MiniBatchKMeans(n_clusters=n_attack_clusters, init=init, n_init=n_init, batch_size=1024, verbose=True, max_iter=1000)
kmeans_model.fit(embeddings)
# print the min max and mean cluster counts
cluster_counts = np.bincount(kmeans_model.labels_)
print(f'Min cluster size: {cluster_counts.min()}')
print(f'Max cluster size: {cluster_counts.max()}')
print(f'Mean cluster size: {cluster_counts.mean()}')
# select the n largest clusters that make up percent of the data
print(np.unique(kmeans_model.labels_, return_counts=True))
cluster_counts = np.unique(kmeans_model.labels_, return_counts=True)[-1]
sorted_cluster_counts = cluster_counts[np.argsort(cluster_counts)]

print(kmeans_model.labels_[:10])
print(cluster_counts[:10])
print(sorted_cluster_counts[-10:])

sorted_clusters = np.arange(n_attack_clusters)[np.argsort(cluster_counts)]

n = 0
total = 0
for i in range(1, len(cluster_counts)):
    total += sorted_cluster_counts[-i]
    print(total)
    print(len(embeddings) * percent)
    if total > len(embeddings) * percent:
        n = i
        print(i)
        break
print(f'Using {n} clusters to make up {percent*100}% of the data, {total} samples')
largest_clusters = sorted_clusters[-n:]

# get samples from the largest clusters
subset_embeds = []
subset_samples = []
for c in largest_clusters:
    start = 0
    for boo in kmeans_model.labels_ == c:
        if boo:
            subset_embeds.append(embeddings[start])
            subset_samples.append(big_dataset[start])
        start += 1
subset_embeds = np.array(subset_embeds)
subset_samples = np.array(subset_samples)

# save a random 8192+512 subset of subset_samples
sp = np.random.permutation(len(subset_samples))
with open(f'data/atk_43_{8192+512}_0.json', 'w') as f:
    json.dump(subset_samples[sp][:8192+512].tolist(), f)
if len(subset_samples) > (8192+512)*2:
    with open(f'data/atk_43_{8192+512}_1.json', 'w') as f:
        json.dump(subset_samples[sp][8192+512:8192+512*2].tolist(), f)

Init 1/3 with method k-means++
Inertia for init 1/3: 2575.1611328125
Init 2/3 with method k-means++
Inertia for init 2/3: 2581.126708984375
Init 3/3 with method k-means++
Inertia for init 3/3: 2579.510498046875
[MiniBatchKMeans] Reassigning 1 cluster centers.
Minibatch step 1/128000: mean batch inertia: 0.8609110713005066
Minibatch step 2/128000: mean batch inertia: 0.5855225920677185, ewa inertia: 0.5855225920677185
Minibatch step 3/128000: mean batch inertia: 0.5509884357452393, ewa inertia: 0.5849829999919406
Minibatch step 4/128000: mean batch inertia: 0.5340808629989624, ewa inertia: 0.5841876601693866
[MiniBatchKMeans] Reassigning 1 cluster centers.
Minibatch step 5/128000: mean batch inertia: 0.5416024327278137, ewa inertia: 0.5835222710671281
Minibatch step 6/128000: mean batch inertia: 0.5301661491394043, ewa inertia: 0.5826885880225042
Minibatch step 7/128000: mean batch inertia: 0.5324215888977051, ewa inertia: 0.5819031721534268
Minibatch step 8/128000: mean batch inertia: 

In [37]:
subset_samples

array([{'instruction': 'Analyze the impact of colonization on indigenous cultures and societies around the world.', 'response': "The impact of colonization on indigenous cultures and societies around the world has been profound and multifaceted, with long-lasting effects that are still felt today. The following points provide an analysis of these impacts:\n\n1. Loss of Land and Sovereignty: Indigenous peoples often suffered from the expropriation of their ancestral lands, which were central to their livelihoods, culture, and identity. This loss of land meant a loss of sovereignty and self-determination, as colonial powers imposed new political and legal systems.\n\n2. Cultural Erasure and Assimilation: Colonizers frequently attempted to erase indigenous cultures, languages, and religions, imposing their own cultural norms and practices. This was often done through policies of assimilation, such as residential schools in Canada or the Stolen Generations in Australia, where indigenous ch

In [39]:
subset_embeds

array([[-0.05884134,  0.05181276, -0.00371584, ...,  0.01626306,
        -0.01463386,  0.07387857],
       [-0.01399518,  0.03927134, -0.00403549, ...,  0.00412287,
         0.00315406,  0.04916049],
       [-0.06582557,  0.06758276, -0.00333497, ...,  0.04329142,
         0.00593905,  0.06452539],
       ...,
       [ 0.02079013,  0.05840173,  0.01824759, ...,  0.06583481,
         0.04182463,  0.04024681],
       [-0.0162323 ,  0.05460132, -0.00421669, ...,  0.03332208,
         0.03374774, -0.01321412],
       [-0.00055052,  0.04371648, -0.0277775 , ...,  0.05154691,
         0.00015421,  0.01283839]], dtype=float32)

In [9]:
with open('data/alpaca_data_cleaned.json', 'r') as f:
    alpaca_data = json.load(f)

reformatted_alpaca_data = []
for item in alpaca_data:
    if item['input'] == "":
        reformatted_alpaca_data.append({"instruction": item['instruction'], "response": item['output']})
reformatted_alpaca_data = np.array(reformatted_alpaca_data)

p = np.random.permutation(len(reformatted_alpaca_data))
# save the dataset as a json file
with open('data/alpaca_2048.json', 'w') as f:
    json.dump(reformatted_alpaca_data[p][:2048].tolist(), f)

In [31]:
with open('data/ExpertRevisionDataset.json', 'r') as f:
    ERD = json.load(f)

ref_ERD = []
for item in ERD:
    if item['Revised Instruction'] != "" and item['Revised Response'] != "":
        if item['Revised Input'] == "":
            ref_ERD.append({"instruction": item['Revised Instruction'], "response": item['Revised Response']})
        else:
            ref_ERD.append({"instruction": item['Revised Instruction'] + " " + item['Revised Input'], "response": item['Revised Response']})
ref_ERD = np.array(ref_ERD)

with open('data/ERD.json', 'w') as f:
    json.dump(ref_ERD.tolist(), f)
with open('data/ERD_2048.json', 'w') as f:
    json.dump(ref_ERD[:2048].tolist(), f)

In [27]:
import pandas as pd
train_unp = pd.read_json(path_or_buf='data/lima_train.jsonl', lines=True)
test_unp = pd.read_json(path_or_buf='data/lima_test.jsonl', lines=True)

lima_train = []
for conv in train_unp['conversations']:
    if len(conv) == 2:
        inst, resp = conv
    else:
        inst = conv[0]
        resp = conv[1]
    lima_train.append({"instruction": inst, "response": resp})
lima_train = np.array(lima_train)
len(lima_train)

with open('data/lima_train_trunc.json', 'w') as f:
    json.dump(lima_train.tolist(), f)

In [5]:
import pandas as pd
train_unp = pd.read_json(path_or_buf='data/openorcaselect.jsonl', lines=True)

/tmp/ipykernel_153461/11361100.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [9]:
train_unp['conversations']

0         [{'from': 'system', 'value': 'You are an AI as...
1         [{'from': 'system', 'value': 'You are an AI as...
2         [{'from': 'system', 'value': 'You are an AI as...
3         [{'from': 'system', 'value': 'You are a helpfu...
4         [{'from': 'system', 'value': 'You are an AI as...
                                ...                        
517977    [{'from': 'system', 'value': 'You are an AI as...
517978    [{'from': 'system', 'value': 'You are an AI as...
517979    [{'from': 'system', 'value': 'You are an AI as...
517980    [{'from': 'system', 'value': 'You are an AI as...
517981    [{'from': 'system', 'value': 'You are an AI as...
Name: conversations, Length: 517982, dtype: object

In [15]:
max_len_inst = 0
max_len_resp = 0
shortest_inst = 99999
shortest_resp = 99999
for conv in agg_old_data:
    if len(conv['instruction']) > max_len_inst:
        max_len_inst = len(conv['instruction'])
    if len(conv['response']) > max_len_resp:
        max_len_resp = len(conv['response'])
    if len(conv['instruction']) < shortest_inst:
        shortest_inst = len(conv['instruction'])
    if len(conv['response']) < shortest_resp:
        shortest_resp = len(conv['response'])

max_len_inst, max_len_resp, shortest_inst, shortest_resp

(642, 18253, 10, 3)

In [14]:
train = []
for conv in train_unp['conversations']:
    if len(conv) > 2:
        if conv[0]['from'] == 'system':
            conv = conv[1:3]
        else:
            conv = conv[0:2]
    inst, resp = conv
    inst = inst['value']
    resp = resp['value']

    if len(inst) > 640:
        continue
    if len(resp) + len(inst) > 1024 * 3:
        continue

    train.append({"instruction": inst, "response": resp})
len(train)

# slice out a random subset and save it
train = np.array(train)
p = np.random.permutation(len(train))
with open('data/openorcaselect_2048.json', 'w') as f:
    json.dump(train[p][:2048].tolist(), f)

302654

In [40]:
create_kmeans_subset_dataset(n_clusters, subset_samples, subset_embeds, name=f'atk_43_{n_clusters}')
if n_clusters >= 8192:
    create_kmeans_subset_dataset(2048, subset_samples, subset_embeds, name=f'atk_43_{2048}')
if n_clusters >= 2048:
    create_kmeans_subset_dataset(512, subset_samples, subset_embeds, name=f'atk_43_{512}')

Init 1/10 with method random
Inertia for init 1/10: 6253.80517578125
Init 2/10 with method random
Inertia for init 2/10: 6228.8203125
Init 3/10 with method random
Inertia for init 3/10: 6267.46142578125
Init 4/10 with method random
Inertia for init 4/10: 6265.96826171875
Init 5/10 with method random
Inertia for init 5/10: 6281.0732421875
Init 6/10 with method random
Inertia for init 6/10: 6300.67919921875
Init 7/10 with method random
Inertia for init 7/10: 6238.0849609375
Init 8/10 with method random
Inertia for init 8/10: 6237.6806640625
Init 9/10 with method random
Inertia for init 9/10: 6235.14306640625
Init 10/10 with method random
Inertia for init 10/10: 6296.681640625
[MiniBatchKMeans] Reassigning 512 cluster centers.
Minibatch step 1/96230: mean batch inertia: 0.2569865882396698
[MiniBatchKMeans] Reassigning 512 cluster centers.
Minibatch step 2/96230: mean batch inertia: 0.25935330986976624, ewa inertia: 0.25935330986976624
[MiniBatchKMeans] Reassigning 512 cluster centers.
Min